In [7]:
%pwd

u'/home/matej/prace/ferda/ferda_regtrack'

In [2]:
%cd ../..

/home/matej/prace/ferda/ferda_regtrack


In [8]:
from core.project.project import Project
from sys import maxsize
import numpy as np
import time
from utils.gt.gt import GT
from utils.gt.gt_project import GtProjectMixin
import tqdm
from core import global_cardinality_classifier as g_cardinality_clf

In [9]:
p = Project('test/project/Sowbug3_cut_300_frames')
gt_path = 'data/GT/Sowbug3_cut.txt'
# p = Project('../projects/2_temp/Camera3-5min/190719_2239')
# gt_path = 'data/GT/Camera3-5min.mp4.txt'

# p = Project('../projects/2_temp/Cam1_clip/190802_1621/')
# gt_path = 'data/GT/Cam1_clip.avi.txt'

loading project: 100%|██████████| 5/5 [00:00<00:00, 16.72it/s]


In [10]:
class GtProject(GtProjectMixin, GT):
    pass

correct_labels = {}
gt = GtProject(filename=gt_path)
gt.set_project_offsets(p)

In [11]:
# construct mapping "tracklet id -> number of objects" from GT
for tracklet in tqdm.tqdm_notebook(p.chm.tracklet_gen(), total=len(p.chm)):
    id_set = gt.tracklet_id_set(tracklet, p)
    if id_set is not None:
        correct_labels[tracklet.id()] = len(id_set)
    else:
        correct_labels[tracklet.id()] = None   

100%|██████████| 5928/5928 [00:05<00:00, 1027.49it/s]


In [7]:
correct_labels

{1: 1,
 2: 1,
 4: 2,
 5: 1,
 346: 1,
 347: 1,
 502: 1,
 503: 1,
 693: 1,
 694: 1,
 737: 1,
 738: 1,
 747: 1,
 749: 1,
 798: 1,
 799: 1,
 827: 1,
 828: 1}

In [12]:
reload(g_cardinality_clf)
collateral_sets = g_cardinality_clf.get_collateral_sets(p, max_frame=5000)
tracklets = set([t for cs in collateral_sets for t in cs])
print("#unique tracklets: {}, #collateral sets: {}".format(len(tracklets), len(collateral_sets)))

#unique tracklets: 18, #collateral sets: 36


In [33]:
# compute cardinalities using ILP
t = time.time()
predecessors = g_cardinality_clf.generate_predecessor_map(tracklets, p)
successors = g_cardinality_clf.generate_successor_map(tracklets, p)
print("predecessors t: {}".format(time.time() - t))

num_animals = len(p.animals)
t = time.time()
areas = g_cardinality_clf.get_median_areas(tracklets, p)
# TODO: medain_area should probably be median from all areas...
median_area = np.median(areas.values())
print("area median extraction time: {}".format(time.time() - t))
area_relaxation_coef = 0

print("#animals {}, #tracklets: {}, #collateral sets: {}, mean area: {}".format(num_animals, len(tracklets), len(collateral_sets), median_area))

cardinalities = g_cardinality_clf.build_ilp_and_solve(tracklets, collateral_sets, predecessors, successors, num_animals, 
                                                      median_area, areas, 
                                                      print_ilp=False)  # area_relaxation_coef=area_relaxation_coef, 

predecessors t: 0.24388217926
area median extraction time: 2.73370909691
#animals 6, #tracklets: 719, #collateral sets: 1438, mean area: 925.0
gamma: 2.0
var registration t: 0.048476934433
objective function: 92.4074997902
constraints: 11.8778829575
ILP construction tooks: 104.348696947s
('Status:', 'Optimal')
('Total Cost = ', 4346481.0)
time: 2.42585492134s
1


In [34]:
import copy
# ants3_tracklets = copy.copy(tracklets)
# ants3_cardinalities = copy.copy(cardinalities)

# sowbug3_tracklets = copy.copy(tracklets)
# sowbug3_cardinalities = copy.copy(cardinalities)

# zebrafish_tracklets = copy.copy(tracklets)
# zebrafish_cardinalities = copy.copy(cardinalities)
# correct_labels[7586] = 1
# correct_labels[4368] = 1
# correct_labels[4349] = 1
# correct_labels[4438] = 1

In [40]:
# eval ILP results against tracklet cardinalities from GT
tracklets_len = {t.id(): len(t) for t in tracklets}

# for coef in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
for coef in [0]:
#     cardinalities = g_cardinality_clf.build_ilp_and_solve(tracklets, collateral_sets, predecessors, successors, num_animals,
#                                                           median_area, areas) # , area_relaxation_coef=coef)
    num_mistakes = 0
    
    tracklets_len_sum = 0
    
    for id in correct_labels.keys():
        if correct_labels[id] is not None and correct_labels[id] != cardinalities[id]:
            print("mistake, id: {}, correct cardinality: {}, estimated: {}".format(id, correct_labels[id], cardinalities[id]))
            num_mistakes += 1
            tracklets_len_sum += tracklets_len[id]
            
    print("COEF: {}, #mistakes: {}, len: {}".format(coef, num_mistakes, tracklets_len_sum))
    print("------------------------------\n")

mistake, id: 11201, correct cardinality: 0, estimated: 1
COEF: 0, #mistakes: 1, len: 1
------------------------------



In [46]:
# eval old cardinality classifier results stored in the project against tracklet cardinalities from GT
num_mistakes = 0
tracklets_len_sum = 0

for t in tracklets:
    try:
        card = correct_labels[t.id()]
        if card is None:
            continue
        if card == 0 and t.is_noise():
            pass
        elif card == 1 and t.is_single():
            pass
        elif card > 1 and t.is_multi():
            pass
        else:
            print("mistake, id: {}, correct cardinality: {}, estimated: {}".format(t.id(), correct_labels[t.id()], t.segmentation_class_str()))
            num_mistakes += 1
            tracklets_len_sum += len(t)
    except:
        print(t.id())
    
print("#mistakes: {}, len: {}".format(num_mistakes, tracklets_len_sum))
print("------------------------------\n")

mistake, id: 3214, correct cardinality: 2, estimated: single
mistake, id: 8311, correct cardinality: 2, estimated: single
mistake, id: 11201, correct cardinality: 0, estimated: single
mistake, id: 11202, correct cardinality: 0, estimated: single
mistake, id: 526, correct cardinality: 2, estimated: single
mistake, id: 9396, correct cardinality: 2, estimated: single
mistake, id: 11495, correct cardinality: 2, estimated: single
mistake, id: 9526, correct cardinality: 2, estimated: single
mistake, id: 1890, correct cardinality: 2, estimated: single
mistake, id: 194, correct cardinality: 2, estimated: single
mistake, id: 195, correct cardinality: 2, estimated: single
mistake, id: 9853, correct cardinality: 2, estimated: single
mistake, id: 8785, correct cardinality: 2, estimated: single
mistake, id: 14398, correct cardinality: 2, estimated: single
mistake, id: 105, correct cardinality: 2, estimated: single
#mistakes: 15, len: 34
------------------------------

